In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from fiona.crs import from_epsg
import urllib, json, requests 
import geojson

In [2]:
from ipyleaflet import Map, GeoData, GeoJSON, basemaps, basemap_to_tiles, Icon, Circle, Marker, LayerGroup, WidgetControl
import ipywidgets as widgets
from ipywidgets import Button 
from IPython.display import display, clear_output

button = Button(description="Generate Text")
output = widgets.Output()

In [3]:
def find_location():
    lat = '40.7210907'
    lon = '-73.9877836'
    
    #Contruct request URL
    url = f'https://geo.fcc.gov/api/census/block/find?'

    # api call using variables instead of directly is way easier
    data_url = f'{url}lat={lat}&lon={lon}&format=json'
    #Get response from API
    response=requests.get(data_url)

    #Parse json in response
    data = response.json()
#     print(response.text)

    #Extracting the county and state being studied 
    state = data['results'][0]['state_fips']
    county_fips = data['results'][0]['county_fips']

    #Separating out the county fips from the state
    county = county_fips[2:]

#     print(state, county)
    
    return state

In [4]:
def download_acs():
    find_location()
    
    state = find_location()
    
    api_key = '9330dc4bf086a84f19fb412bb15f232507301de6'
    acs_url = f'https://api.census.gov/data/2018/acs/acs5/'

    pop_sex = 'B01003_001E,B01001_002E,B01001_026E'
    household_type = 'B11001_002E,B11001_007E'
    education = 'B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E'
    household_income = 'B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,B19001_010E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E'
    means_of_transport = 'B08301_002E,B08301_003E,B08301_003E,B08301_010E,B08301_016E,B08301_017E,B08301_018E,B08301_019E,B08301_020E,B08301_021E'
    travel_time = 'B08303_002E,B08303_003E,B08303_004E,B08303_005E,B08303_006E,B08303_007E,B08303_008E,B08303_009E,B08303_010E,B08303_011E,B08303_012E,B08303_013E'

    # api call using variables instead of directly is way easier
    pop_sex_url = f'{acs_url}?&get={pop_sex}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
    household_type_url = f'{acs_url}?&get={household_type}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
    education_url = f'{acs_url}?&get={education}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
    household_income_url = f'{acs_url}?&get={household_income}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
    means_of_transport_url = f'{acs_url}?&get={means_of_transport}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
    travel_time_url = f'{acs_url}?&get={travel_time}&for=tract:*&in=state:{state}%20county:*&key={api_key}'

    response_acs1=requests.get(pop_sex_url)
    response_acs2=requests.get(household_type_url)
    response_acs3=requests.get(education_url)
    response_acs4=requests.get(household_income_url)
    response_acs5=requests.get(means_of_transport_url)
    response_acs6=requests.get(travel_time_url)
    # print(response_acs.text)

    #load the response into a JSON
    data_acs1=response_acs1.json()
    data_acs2=response_acs2.json()
    data_acs3=response_acs3.json()
    data_acs4=response_acs4.json()
    data_acs5=response_acs5.json()
    data_acs6=response_acs6.json()

    #ignoring the first element which is column names, replacing columns with the columns from the table
    acs1=pd.DataFrame(data_acs1[1:], columns=data_acs1[0])
    acs2=pd.DataFrame(data_acs2[1:], columns=data_acs2[0])
    acs3=pd.DataFrame(data_acs3[1:], columns=data_acs3[0])
    acs4=pd.DataFrame(data_acs4[1:], columns=data_acs4[0])
    acs5=pd.DataFrame(data_acs5[1:], columns=data_acs5[0])
    acs6=pd.DataFrame(data_acs6[1:], columns=data_acs6[0])
    
    return acs1, acs2, acs3, acs4, acs5, acs6

In [5]:
# def import_censustracts():
#     ##Adding in census geographies
#     state='STATE=36'

#     # census tracts link
#     endpoint = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/4/query'
#     s = requests.session()
#     s.params = {
#         'where': 'STATE=36 AND COUNTY=',
#         'text': '',
#         'objectIds': '',
#         'time': '',
#         'geometry': '[[-2336502.536,-20037700.],[1.1615796196E7,2.0656504635E7]]',
#         'geometryType': 'esriGeometryEnvelope',
#         'inSR': 102100, # EPSG 3857
#         'spatialRel': 'esriSpatialRelIntersects',
#         'relationParam': '',
#     #specify fields
#         'outFields': 'GEOID,STATE,COUNTY,TRACT,STGEOMETRY, OBJECTID',
#         'returnGeometry': True,
#         'returnTrueCurves': False,
#         'maxAllowableOffset': '',
#         'geometryPrecision': '',
#         'outSR': '',
#         'returnIdsOnly': False,
#         'returnCountOnly': False,
#         'orderByFields': '',
#         'groupByFieldsForStatistics': '',
#         'outStatistics': '',
#         'returnZ': False,
#         'returnM': False,
#         'gdbVersion': '',
#         'returnDistinctValues': False,
#         'f': 'geojson',        
#     }
#     start = 0
#     done = False
#     features = []
#     crs = None
#     while not done:
#         r = s.get(endpoint, params={
#             'resultOffset': start,
#             'resultRecordCount': 32,
#         })
#         r.raise_for_status()
#         tracts = geojson.loads(r.text)
#         newfeats = tracts.__geo_interface__['features']
#         if newfeats:
#             features.extend(newfeats)
#             crs=tracts.__geo_interface__['crs']
#             start += len(newfeats)
#             print("Received", len(newfeats), "entries,", start, "total")
#         else:
#             done = True
#     tracts_df = gpd.GeoDataFrame.from_features(features, crs=crs)
    
#     return tracts_df

In [27]:
display(button, output)

def on_button_clicked(b):
    output.clear_output()
    
    find_location()
    
    global acs1, acs2, acs3, acs4, acs5, acs6
    acs1, acs2, acs3, acs4, acs5, acs6 = download_acs()
#     tracts_df = import_censustracts()
    display(acs1, acs2, acs3, acs4, acs5, acs6) 

    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Generate Text', style=ButtonStyle())

Output(outputs=({'output_type': 'stream', 'text': 'Button clicked.\n', 'name': 'stdout'},))

,B01003_001E,B01001_002E,B01001_026E,state,county,tract
0,3024,1538,1486,36,091,061404
1,3562,1872,1690,36,095,740500
2,2076,1025,1051,36,089,492900
3,4410,2118,2292,36,089,492100
4,6888,3341,3547,36,091,062405
...,...,...,...,...,...,...
4913,3366,1575,1791,36,059,517000
4914,3735,1766,1969,36,059,517801
4915,2846,1419,1427,36,059,518502
4916,5647,2684,2963,36,119,010000


,B11001_002E,B11001_007E,state,county,tract
0,852,277,36,091,061404
1,907,581,36,095,740500
2,499,291,36,089,492900
3,895,571,36,089,492100
4,1840,1445,36,091,062405
...,...,...,...,...,...
4913,851,328,36,059,517000
4914,1012,234,36,059,517801
4915,855,236,36,059,518502
4916,1565,179,36,119,010000


,B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,state,county,tract
0,2104,311,105,145,264,246,639,283,16,42,36,091,061404
1,2604,827,135,229,355,182,323,239,25,11,36,095,740500
2,1410,405,90,113,128,164,102,108,17,12,36,089,492900
3,2661,586,201,145,300,322,413,260,61,66,36,089,492100
4,5209,932,181,369,675,618,1078,781,96,189,36,091,062405
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,2569,517,33,21,188,120,657,452,241,97,36,059,517000
4914,2599,267,10,57,180,140,1063,538,204,61,36,059,517801
4915,2078,304,54,61,201,167,527,475,80,68,36,059,518502
4916,3435,89,31,23,122,30,848,1114,887,262,36,119,010000


...

,B11001_002E,B11001_007E,state,county,tract
0,852,277,36,091,061404
1,907,581,36,095,740500
2,499,291,36,089,492900
3,895,571,36,089,492100
4,1840,1445,36,091,062405
...,...,...,...,...,...
4913,851,328,36,059,517000
4914,1012,234,36,059,517801
4915,855,236,36,059,518502
4916,1565,179,36,119,010000


,B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,state,county,tract
0,2104,311,105,145,264,246,639,283,16,42,36,091,061404
1,2604,827,135,229,355,182,323,239,25,11,36,095,740500
2,1410,405,90,113,128,164,102,108,17,12,36,089,492900
3,2661,586,201,145,300,322,413,260,61,66,36,089,492100
4,5209,932,181,369,675,618,1078,781,96,189,36,091,062405
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,2569,517,33,21,188,120,657,452,241,97,36,059,517000
4914,2599,267,10,57,180,140,1063,538,204,61,36,059,517801
4915,2078,304,54,61,201,167,527,475,80,68,36,059,518502
4916,3435,89,31,23,122,30,848,1114,887,262,36,119,010000


,B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,B19001_010E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E,state,county,tract
0,25,37,10,9,5,31,44,22,69,59,85,207,80,140,169,137,36,091,061404
1,104,108,109,67,49,36,109,78,32,136,144,220,133,42,97,24,36,095,740500
2,65,52,70,43,33,30,47,61,48,115,55,69,47,24,14,17,36,089,492900
3,126,187,86,89,72,39,76,42,54,128,113,178,116,70,77,13,36,089,492100
4,129,107,100,242,88,149,109,257,158,304,188,625,241,186,92,310,36,091,062405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,17,25,33,45,8,10,45,12,35,66,26,135,123,76,192,331,36,059,517000
4914,31,11,14,12,18,15,31,21,18,46,92,82,130,64,108,553,36,059,517801
4915,42,8,37,10,43,67,17,0,11,33,97,170,113,130,173,140,36,059,518502
4916,16,26,0,0,36,0,28,11,0,17,33,75,94,46,117,1245,36,119,010000


,B08301_002E,B08301_003E,B08301_003E,B08301_010E,B08301_016E,B08301_017E,B08301_018E,B08301_019E,B08301_020E,B08301_021E,state,county,tract
0,1547,1402,1402,16,13,0,0,29,0,165,36,091,061404
1,1475,1336,1336,13,0,4,5,50,9,62,36,095,740500
2,649,535,535,0,0,0,0,19,8,98,36,089,492900
3,1238,1048,1048,39,0,11,17,175,9,162,36,089,492100
4,3603,3375,3375,46,0,0,0,144,98,142,36,091,062405
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,1369,1217,1217,110,35,0,3,31,4,78,36,059,517000
4914,1139,1032,1032,195,2,0,0,83,25,231,36,059,517801
4915,1181,1054,1054,251,20,0,0,30,6,34,36,059,518502
4916,1010,985,985,1233,16,0,0,11,17,291,36,119,010000


,B08303_002E,B08303_003E,B08303_004E,B08303_005E,B08303_006E,B08303_007E,B08303_008E,B08303_009E,B08303_010E,B08303_011E,B08303_012E,B08303_013E,state,county,tract
0,9,79,226,362,159,64,100,74,218,192,74,48,36,091,061404
1,49,186,218,233,149,45,149,75,30,205,170,47,36,095,740500
2,65,67,41,87,45,22,116,48,49,56,57,23,36,089,492900
3,102,298,261,196,239,57,168,24,12,50,68,14,36,089,492100
4,148,485,404,419,719,411,557,147,182,270,89,60,36,091,062405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,48,216,170,118,123,120,335,9,85,73,95,160,36,059,517000
4914,82,141,118,166,141,55,174,68,40,120,185,154,36,059,517801
4915,0,96,117,266,182,99,171,59,40,43,237,178,36,059,518502
4916,19,66,195,176,95,40,89,45,88,471,949,54,36,119,010000


In [28]:
acs3.head()

,B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,state,county,tract
0,2104,311,105,145,264,246,639,283,16,42,36,091,061404
1,2604,827,135,229,355,182,323,239,25,11,36,095,740500
2,1410,405,90,113,128,164,102,108,17,12,36,089,492900
3,2661,586,201,145,300,322,413,260,61,66,36,089,492100
4,5209,932,181,369,675,618,1078,781,96,189,36,091,062405


In [25]:
acs1.head()

,B01003_001E,B01001_002E,B01001_026E,state,county,tract
0,3024,1538,1486,36,091,061404
1,3562,1872,1690,36,095,740500
2,2076,1025,1051,36,089,492900
3,4410,2118,2292,36,089,492100
4,6888,3341,3547,36,091,062405


In [ ]:
####Actually, download the census first and do the intersections so you only need to download the census data for relevant places basically 